In [9]:
import pandas as pd
import json

In [38]:
import json
import pandas as pd

row_data = []
file_path = "synthdata/inference_results_oss120b.jsonl"

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            data = json.loads(line)
            row_data.append(data['output_json']['items'])
        except Exception as e:
            pass

# ---- Flatten each row ----
def flatten_label_xai(d):
    out = {}
    for k, v in d.items():
        out[k] = v.get('label')      # e.g. IT10: False
        out[f"{k}_XAI"] = v.get('xai')  # e.g. IT10_XAI: "explanation..."
    return out

# Apply to all rows
flat_data = [flatten_label_xai(r) for r in row_data]

# ---- Load into pandas ----
df = pd.DataFrame(flat_data)

In [ ]:
df = df.replace({True: 1, False: 0})
df = df.reindex(sorted(df.columns), axis=1)

In [ ]:
# create a df_compact in which _XAI columns are dropped
df_compact = df.loc[:, ~df.columns.str.endswith('_XAI')]

In [ ]:
df_compact.to_csv("synthdata/df_compact.csv", index=False)